In [ ]:
from positional_encodings.torch_encodings import PositionalEncoding1D, PositionalEncoding2D, PositionalEncoding3D, Summer
import numpy as np
import torch
import matplotlib.pyplot as plt
from scipy.io import savemat, loadmat
import random
random.seed(2)
torch.manual_seed(2)
torch.cuda.manual_seed(2)
np.random.seed(2)

mdict = loadmat("C:/Users/mehrdad/KSpaceNet/PosEncode.mat")
## TODO: this cell should be saves seperatly and the result could be loaded at the initially
PosEncoding = mdict["PositionalEncoding"]

## Load Postional Condition, Repeat it based on batch_size and Convert it Torch.Tensor
# PosEncoding = scipy.loatmat()
PosEncoding = torch.tensor(PosEncoding)
# PosEncoding = PosEncoding[np.newaxis,:,:,:].repeat(batch_size,1,1,1)
print(PosEncoding.shape)


In [ ]:
import matplotlib.pyplot as plt
import scipy.fftpack
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms, utils
import torchvision
import torch
import glob
from medpy.io import load, save
import sys
from scipy.io import savemat

save_to_dir = 'J:/Anvari/MsThesis/UNET-k2/10_Percent/Gaussian/inference/'
kmodel_dir1 = 'J:/Anvari/MsThesis/K-Space-Net2/T1_model/'
kmodel_dir2 = 'J:/Anvari/MsThesis/K-Space-Net2/T2_model/'
gmodel_dir = 'J:/Anvari/MsThesis/UNET-k2/10_Percent/Gaussian/'
LINE_CLEAR = '\x1b[2K' # <-- ANSI sequence
train_root = 'J:\\Anvari\\proccessed_data_257\\train\\'
validation_root = 'J:\\Anvari\\proccessed_data_257\\test\\'
transform_2015 = transforms.Compose([transforms.ConvertImageDtype(torch.float),
                                     transforms.Normalize(0.5, 0.5, 0.5)])
def load_mask(mask_name):
    dir = 'C:\\Users\\mehrdad\\MS_projects\\mask\\Build mask'
    matfile = scipy.io.loadmat(dir + '\\{}.mat'.format(mask_name)) 
    print(type(matfile))
    print(matfile.keys())
    # print(matfile.get("__header__"))
    mask = np.zeros((6,257,257))
    mask_P = matfile.get('PrimaryMask')
    mask_S = matfile.get('SecondaryMask')
    mask_P = np.logical_or(mask_P,np.flip(mask_P))
    mask_S = np.logical_or(mask_S,np.flip(mask_S))
    mask[0,:,:]=mask_S
    mask[1,:,:]=mask_P
    mask[2,:,:]=mask_S
    mask[3,:,:]=mask_P
    mask[4,:,:]=mask_S
    mask[5,:,:]=mask_P
    print(mask.shape)
    print(mask.dtype)
    print(type(mask))
    plt.imshow(mask_P)
    plt.title(mask_name)
    plt.axis('off')
    plt.show()
    plt.imshow(mask_S)
    plt.title(mask_name)
    plt.axis('off')
    plt.show()
    plt.imshow(np.logical_or(mask_P,mask_S))
    plt.title(mask_name)
    plt.axis('off')
    plt.show()
    return mask

mask = load_mask('GaussianDistribution1DMask_10_257')
# mask = load_mask('UniformDistribution1DMask_30_257')
# mask = load_mask('Deterministic1DMask_40_257')


mask_c = 1 - mask


# def cal_fft_for_all_channels(x):
#     fft = np.zeros_like(x).astype('complex128')
#     for i in range(6):
#         fft[i, :, :] = scipy.fftpack.fft2(x[i, :, :])
#         fft[i, :, :] = scipy.fftpack.fftshift(fft[i, :, :])
#     return np.real(fft).astype('float32'),np.imag(fft).astype('float32')


def cal_fft_for_all_channels(x):
    # fft = torch.zernp.zeros_like(x).astype('complex128'), dtype=torch.complex64
    fft = torch.zeros_like(x,dtype=torch.complex64)
    fft = torch.fft.fft2(x)
    fft = torch.fft.fftshift(fft,dim=(1,2))
    # for i in range(6):
    #     fft[i, :, :] = scipy.fftpack.fft2(x[i, :, :])
    #     fft[i, :, :] = scipy.fftpack.fftshift(fft[i, :, :])
    return torch.real(fft).to(torch.float32) ,torch.imag(fft).to(torch.float32)

def to_bad_img(x, mask):
    x = (x + 1.) / 2.
    for i in range(6):
        fft = scipy.fftpack.fft2(x[i, :, :])
        fft = scipy.fftpack.fftshift(fft)
        fft = fft * mask[i,:,:]
        fft = scipy.fftpack.ifftshift(fft)
        x[:, :, i] = scipy.fftpack.ifft2(fft)
        x[:, :, i] = np.abs(x[:, :, i])
        x[:, :, i] = x[:, :, i] * 2 - 1
    return x


class Brats2013_2D(Dataset):
    def __init__(self, root, transform=transform_2015 , PE = PosEncoding):
        self.img_dir = root
        self.positionalEncoding = PE
        self.transform = transform
        file_list = glob.glob(root + "*.mha")
        self.data = []
        for file_path in file_list:
            layer_name = file_path.split("\\")[-1].split(".")[-2].split("_")[-1]
            self.data.append([file_path, int(layer_name)])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, LayerNum = self.data[idx]
        image, image_header = load(img_path)
        image = torch.tensor(image).cuda().to(torch.float32)
        image_fft_r,image_fft_i = cal_fft_for_all_channels(image)
        image_fft_r = torch.tensor(image_fft_r)
        image_fft_i = torch.tensor(image_fft_i)
        image = torch.tensor(image)
        return image_fft_r, image_fft_i, image, torch.tensor(LayerNum), torch.tensor(self.positionalEncoding)


def imshow(img, batch_size, name, cmap):
    npimg = img.numpy()
    npimg = np.transpose(npimg, (1, 2, 0))
    plt.figure(figsize=(4, 4))
    if cmap != 'gray':
        # c = 1 / np.log(1 + np.max(npimg[:,:,0]))
        # image = c * (np.log(npimg[:,:,0] + 1))
        image = npimg[:,:,0]
    else:
        image = npimg[:,:,0]

    # cm = plt.get_cmap(cmap)
    # npimg = cm(image)
    if cmap != 'gray':
        image[-1,-1] = 0.1
        image[-2,-1] = 0
        plt.imshow(image,interpolation = 'none',cmap=cmap,vmin=0,vmax=0.1)
        plt.colorbar(orientation="vertical")
        plt.axis('off')
        plt.savefig(save_to_dir+'{}_01.png'.format(name), format='png', dpi=600)
        plt.figure(figsize=(4, 4))
        image[-1,-1] = 1
        image[-2,-1] = 0
        plt.imshow(image,interpolation = 'none',cmap=cmap,vmin=0,vmax=1)
        plt.colorbar(orientation="vertical")
        plt.axis('off')
        plt.savefig(save_to_dir+'{}_10.png'.format(name), format='png', dpi=600)
    else:
        image[-1,-1] = 1
        image[-2,-1] = 0
        plt.imshow(image,interpolation = 'none',cmap=cmap,vmin=0,vmax=1)
        plt.axis('off')
        plt.savefig(save_to_dir+'{}.png'.format(name), format='png', dpi=600)
        
    
    # plt.title('Batch-size = {}'.format(batch_size))
    


def imsave(img,epoch,i,name,batch_size):
    img_slices = np.zeros((6*batch_size,240,240))
    for j in range(batch_size*6):
        img_slices[j,:,:] = img[int(j//6),int(j%6),:,:]
    save(img_slices,save_to_dir+f'{name}_{epoch}_{i}.mha')
    
batch_size = 10
# dataset = Brats2013_2D(root=train_root, PE= PosEncoding)
dataset_val = Brats2013_2D(root=validation_root , PE= PosEncoding)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=False,drop_last=True)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True,pin_memory=False,drop_last=False)

In [ ]:
import torch.nn.functional as F
import torch
from torch import nn
import numpy as np
from torchsummary import summary
from torchvision import models
if torch.cuda.is_available():
    print('it works')
else:
    print('it doesnt work')
device = torch.device("cuda:0" if (torch.cuda.is_available() and 1 > 0) else "cpu")
print('device is {}'.format(device))

class u_net_bn(nn.Module):
    def __init__(self,ngpu):
        super(u_net_bn, self).__init__()
        self.ngpu = ngpu
        self.df_dim = 64
        self.conv1 = nn.Conv2d(2, self.df_dim, (4, 4), (2, 2), padding=(1, 1))
        self.conv2 = nn.Conv2d(1 * self.df_dim, 2 * self.df_dim, (4, 4), (2, 2), padding=(1, 1))
        self.conv3 = nn.Conv2d(2 * self.df_dim, 4 * self.df_dim, (4, 4), (2, 2), padding=(1, 1))
        self.conv4 = nn.Conv2d(4 * self.df_dim, 8 * self.df_dim, (4, 4), (2, 2), padding=(1, 1))
        self.conv5 = nn.Conv2d(8 * self.df_dim, 8 * self.df_dim, (4, 4), (2, 2), padding=(1, 1))
        self.conv6 = nn.Conv2d(8 * self.df_dim, 8 * self.df_dim, (4, 4), (2, 2), padding=(1, 1))
        self.conv7 = nn.Conv2d(8 * self.df_dim, 8 * self.df_dim, (4, 4), (2, 2), padding=(1, 1))
        self.conv8 = nn.Conv2d(8 * self.df_dim, 8 * self.df_dim, (2, 2), (2, 2))
        self.up7 = nn.ConvTranspose2d(8 * self.df_dim, 8 * self.df_dim, (2, 2), stride=(2, 2), padding=(0, 0))
        self.up6 = nn.ConvTranspose2d(16 * self.df_dim, 16 * self.df_dim, (4, 4), (2, 2), padding=(1, 1))
        self.up5 = nn.ConvTranspose2d(24 * self.df_dim, 16 * self.df_dim, (4, 4), (2, 2), padding=(1, 1))
        self.up4 = nn.ConvTranspose2d(24 * self.df_dim, 16 * self.df_dim, (4, 4), (2, 2), padding=(1, 1))
        self.up3 = nn.ConvTranspose2d(24 * self.df_dim, 4 * self.df_dim, (4, 4), (2, 2), padding=(1, 1))
        self.up2 = nn.ConvTranspose2d(8 * self.df_dim, 2 * self.df_dim, (4, 4), (2, 2), padding=(1, 1))
        self.up1 = nn.ConvTranspose2d(4 * self.df_dim, 1 * self.df_dim, (4, 4), (2, 2), padding=(1, 1))
        self.up0 = nn.ConvTranspose2d(2 * self.df_dim, 1 * self.df_dim, (4, 4), (2, 2), padding=(1, 1))

        self.batch_conv2 = nn.BatchNorm2d(2 * self.df_dim)
        self.batch_conv3 = nn.BatchNorm2d(4 * self.df_dim)
        self.batch_conv4 = nn.BatchNorm2d(8 * self.df_dim)
        self.batch_conv5 = nn.BatchNorm2d(8 * self.df_dim)
        self.batch_conv6 = nn.BatchNorm2d(8 * self.df_dim)
        self.batch_conv7 = nn.BatchNorm2d(8 * self.df_dim)
        self.batch_up0 = nn.BatchNorm2d(1 * self.df_dim)
        self.batch_up1 = nn.BatchNorm2d(1 * self.df_dim)
        self.batch_up2 = nn.BatchNorm2d(2 * self.df_dim)
        self.batch_up3 = nn.BatchNorm2d(4 * self.df_dim)
        self.batch_up4 = nn.BatchNorm2d(16 * self.df_dim)
        self.batch_up5 = nn.BatchNorm2d(16 * self.df_dim)
        self.batch_up6 = nn.BatchNorm2d(16 * self.df_dim)
        self.batch_up7 = nn.BatchNorm2d(8 * self.df_dim)

        self.outLayer = nn.Conv2d(self.df_dim, 1, (1, 1))

    def forward(self, x):
        x = self.conv1(x)
        c1 = nn.LeakyReLU(0.2)(x)
#         print(f'c1:{c1.shape}')
        
        x = self.conv2(x)
        x = self.batch_conv2(x)
        c2 = nn.LeakyReLU(0.2)(x)
#         print(f'c2:{c2.shape}')
        
        x = self.conv3(c2)
        x = self.batch_conv3(x)
        c3 = nn.LeakyReLU(0.2)(x)
#         print(f'c3:{c3.shape}')
        
        x = self.conv4(x)
        x = self.batch_conv4(x)
        c4 = nn.LeakyReLU(0.2)(x)
#         print(f'c4:{c4.shape}')
        
        x = self.conv5(c4)
        x = self.batch_conv5(x)
        c5 = nn.LeakyReLU(0.2)(x)
#         print(f'c5:{c5.shape}')
        
        x = self.conv6(c5)
        x = self.batch_conv6(x)
        c6 = nn.LeakyReLU(0.2)(x)
#         print(f'c6:{c6.shape}')
        c7 = self.conv7(c6)
        c7 = self.batch_conv7(c7)
#         print(f'c7:{c7.shape}')
        x = nn.LeakyReLU(0.2)(c7)
        x = self.conv8(x)
        # x = self.batch_conv8(x)
        # print(f'c8:{x.shape}')
        u7 = self.up7(x)
        u7 = self.batch_up7(u7)
        u7 = nn.ReLU()(u7)
#         print(f'u7:{u7.shape}')
        x = torch.concat((u7, c7), dim=1)
#         print(f'x:{x.shape}')
        x = self.up6(x)
        u6 = self.batch_up6(x)
        u6 = nn.ReLU()(u6)
#         print(f'u6:{u6.shape}')
        x = torch.concat((u6, c6), dim=1)
        x = self.up5(x)
        x = self.batch_up5(x)
        u5 = nn.ReLU()(x)
#         print(f'u5:{u5.shape}')
    
        x = torch.concat((u5, c5), dim=1)
        x = self.up4(x)
        x = self.batch_up4(x)
        u4 = nn.ReLU()(x)
#         print(f'u4:{u4.shape}')
        
        x = torch.concat((u4, c4), dim=1)
        x = self.up3(x)
        x = self.batch_up3(x)
        u3 = nn.ReLU()(x)
#         print(f'u3:{u3.shape}')
        
        x = torch.concat((u3, c3), dim=1)
        x = self.up2(x)
        x = self.batch_up2(x)
        u2 = nn.ReLU()(x)
        
        x = torch.concat((u2, c2), dim=1)
        x = self.up1(x)
        x = self.batch_up1(x)
        u1 = nn.ReLU()(x)
        
        x = torch.concat((u1, c1), dim=1)
        x = self.up0(x)
        x = self.batch_up0(x)
        p2d = (0, 1, 0, 1) 
        x = F.pad(x, p2d, "constant", 0)
        x = nn.ReLU()(x)

        x = self.outLayer(x)
        x = nn.Tanh()(x)
        # print(f'u3:{x.shape}')
        return x

netG = u_net_bn(1).to(device)
summary(netG, input_size=(2, 257, 257))


In [ ]:
from torch import nn
import math
class SinusoidalPositionEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, layer):
        device = layer.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = layer[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings


In [ ]:
import torch.nn.functional as F
import torch
from torch import nn
import numpy as np
from torchsummary import summary
from torchvision import models
from einops import rearrange
if torch.cuda.is_available():
    print('it works')
else:
    print('it doesnt work')
device = torch.device("cuda:0" if (torch.cuda.is_available() and 1 > 0) else "cpu")
print('device is {}'.format(device))


class KspaceNetT1(nn.Module):
    def __init__(self,ngpu):
        super(KspaceNetT1, self).__init__()
        self.df_dim = 32     ## TODO: normalizing the input k-space
        self.posEn_dim = 34
        self.layEn_dim = 32
        self.conv1 = nn.Conv2d(10 + self.posEn_dim + self.layEn_dim , 4*self.df_dim, (1, 1), padding=(0, 0))
        self.conv2 = nn.Conv2d(4*self.df_dim, 2*self.df_dim, (1, 1), padding=(0, 0))
        self.conv3 = nn.Conv2d(2*self.df_dim, 1*self.df_dim, (1, 1), padding=(0, 0))
        self.conv4 = nn.Conv2d(1*self.df_dim, 2, (1, 1), padding=(0, 0))

        self.batch1 = nn.BatchNorm2d(4 * self.df_dim)
        self.batch2 = nn.BatchNorm2d(2 * self.df_dim)
        self.batch3 = nn.BatchNorm2d(1 * self.df_dim)

        ## TODO: an MLP model for Layer encoding 
        self.layer_mlp = nn.Sequential(
            SinusoidalPositionEmbeddings(self.layEn_dim),
            nn.Linear(self.layEn_dim, self.layEn_dim),
            nn.GELU(),
            nn.Linear(self.layEn_dim, self.layEn_dim),
        )
        
        self.layer_mlp2 = nn.Sequential(
            SinusoidalPositionEmbeddings(self.layEn_dim),
            nn.Linear(self.layEn_dim, self.layEn_dim),
            nn.GELU(),
            nn.Linear(self.layEn_dim, 2*self.posEn_dim),
        )
        
        self.layer_mlp3 = nn.Sequential(
            SinusoidalPositionEmbeddings(self.layEn_dim),
            nn.Linear(self.layEn_dim, self.layEn_dim),
            nn.GELU(),
            nn.Linear(self.layEn_dim, 10),
        )
        
        self.convLEPEforX = nn.Conv2d(self.posEn_dim + 10, 10*2, (1, 1), padding=(0, 0))
        
    def forward(self, x, PE, LE):

        layerEmbeddingLatent = self.layer_mlp(LE)
        
        LEchs = rearrange(layerEmbeddingLatent, "b c -> b c 1 1")
        LEchs = LEchs.repeat(1,1,257,257) ## TODO: check if scale and shifting X using LEchs is any better
        ## TODO: Check if scale and shifting PE using LEch is any better
        LEforPE = self.layer_mlp2(LE)
        LEforPE2 = rearrange(LEforPE, "b c -> b c 1 1")
        scale_shift = LEforPE2.chunk(2, dim=1)
        scale, shift = scale_shift
        PE = PE * (scale + 1) + shift
        ## TODO: Check if scale and shiftinf PE using LEch and concat it to X while the X itself is scaled and shifted by scaled and shifted PE 
        LEforX = self.layer_mlp3(LE)
        LEforX2 = rearrange(LEforX, "b c -> b c 1 1")
        LEforX2 = LEforX2.repeat(1,1,257,257)
        LEPEforX = torch.concat((PE, LEforX2),dim=1)
        LEPEforX2 = self.convLEPEforX(LEPEforX)
        scale_shift = LEPEforX2.chunk(2, dim=1)
        scale, shift = scale_shift
        x = x * (scale + 1) + shift
        ## TODO: Concatinate input (x), Positional Encoding (PE), Layer Encoding (LE)
        
        
        x = torch.concat((x, PE, LEchs),dim=1)

        x = self.conv1(x)
        x = nn.LeakyReLU(0.2)(x)
        x = self.batch1(x)

        x = self.conv2(x)
        x = nn.LeakyReLU(0.2)(x)
        x = self.batch2(x)

        x = self.conv3(x)
        x = nn.LeakyReLU(0.2)(x)
        x = self.batch3(x)

        x = self.conv4(x)
#         x = nn.Tanh()(x)
        
        return x
    

In [ ]:
import os.path
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils

ngpu = 1
lr = 0.0001
lr_decay = 0.5
decay_every = 5
n_epoch = 6
beta1 = 0.5

print(torch.__version__)
if torch.cuda.is_available():
    print('it works')
else:
    print('it doesnt work')
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print('device is {}'.format(device))
# torch.cuda.memory_summary(device=None, abbreviated=True)
torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=None, abbreviated=False)


# print("Defining Dataloader...")
batch_size = 10
workers = 1



In [ ]:
def infer_using_KspaceNet(image_fft_r,image_fft_i,LayerNum,model,mask,mask_c,t_c,t_p,PosEncoding):
    for i in range(6):
            image_fft_r[:,i,:,:] = image_fft_r[:,i,:,:].to(device) * mask[i,:,:]
            image_fft_i[:,i,:,:] = image_fft_i[:,i,:,:].to(device) * mask[i,:,:]
    ListConChIdxs = []
    if t_c == 1:
        ListConChIdxs = [0,t_c,2,3,5] ## index 1 is the T1 middle layer that should be predicted
    if t_c == 4:
        ListConChIdxs = [0,2,3,t_c,5]
    image_fft_in = torch.concat((image_fft_r[:,ListConChIdxs,:,:],image_fft_i[:,ListConChIdxs,:,:]), dim = 1).to(device)
    image_fft_out = torch.concat((image_fft_r[:,t_p,:,:][:,np.newaxis,:,:],image_fft_i[:,t_p,:,:][:,np.newaxis,:,:]), dim = 1).to(device)
    ## Inference
    Predition = model(image_fft_in.float(),PosEncoding.float(),LayerNum.to(device))
    
    # correct values of estimated image using true ones
    Predition[:,0,:,:] = Predition[:,0,:,:] * mask_c[t_p,:,:] + image_fft_r[:,t_p,:,:].to(device) * mask[t_p,:,:]
    Predition[:,1,:,:] = Predition[:,1,:,:] * mask_c[t_p,:,:] + image_fft_i[:,t_p,:,:].to(device) * mask[t_p,:,:]
    
    complexPrediction  = Predition[:,0,:,:][:,np.newaxis,:,:]+Predition[:,1,:,:][:,np.newaxis,:,:]*1j
    complexPrediction = torch.fft.ifftshift(complexPrediction,dim=(2,3))
    image_Predition = torch.fft.ifft2(complexPrediction)
    image_Predition = torch.real(torch.tensor(image_Predition).float())
    return image_Predition

In [ ]:
def cal_losses(fake1,good_imgs1,Pred_slice):
    fake_fft2 = torch.fft.fft2(fake1)
    good_img_fft2 = torch.fft.fft2(good_imgs1)

    fake_fft2 = torch.fft.fftshift(fake_fft2) * mask_c[Pred_slice,:,:] + torch.fft.fftshift(good_img_fft2) * mask[Pred_slice,:,:]
    fake_fft2 = torch.fft.ifftshift(fake_fft2)
    fake1 = torch.fft.ifft2(fake_fft2).float()
    
    errG_fft_mse_r = torch.mean(torch.square(fake_fft2.real - good_img_fft2.real)) * 0.02
    errG_fft_mse_i = torch.mean(torch.square(fake_fft2.imag - good_img_fft2.imag)) * 0.02
    errG_fft = errG_fft_mse_r + errG_fft_mse_i

    errG_numerator = torch.sqrt(torch.sum(torch.square(fake1 - good_imgs1)))
    errG_denominator = torch.sqrt(torch.sum(torch.square(good_imgs1)))
    errG_mse = (errG_numerator/errG_denominator)*15
    
    return errG_fft, errG_mse, fake1

In [ ]:
from torchmetrics import StructuralSimilarityIndexMeasure, PeakSignalNoiseRatio
import os
import json
from tqdm import tqdm
torch.cuda.empty_cache()

print('netG to device..')
netG = u_net_bn(ngpu).to(device)
netG = netG.float()
## Define the model
model2 = KspaceNetT1(ngpu).to(device)
model1 = KspaceNetT1(ngpu).to(device)

# print('netG weights init..')
# netG.apply(weights_init)

criterion_mse = torch.nn.MSELoss()

# Setup Adam optimizers
print('Setup Adam optimizers for G')
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))
scheduler2 = optim.lr_scheduler.ExponentialLR(optimizerG, gamma=0.5)

print(f'scheduler : {scheduler2.get_last_lr()}')
## Define Metrics
psnr = PeakSignalNoiseRatio().to(device)
ssim = StructuralSimilarityIndexMeasure().to(device)

# psnr = PeakSignalNoiseRatio(dim=(1,2,3),reduction='elementwise_mean',data_range=1).to(device)
# ssim = StructuralSimilarityIndexMeasure(dim=(1,2,3),reduction='elementwise_mean',data_range=1).to(device)


## Load the model if exists, pre-trained model

if  os.path.isfile(gmodel_dir + 'model_unet.pt'):
    netG = torch.load(gmodel_dir + 'model_unet.pt')
    
netG.eval()

if  os.path.isfile(kmodel_dir2 + 'modelt2.pt'):
    model2 = torch.load(kmodel_dir2 + 'modelt2.pt')
    
model2.eval()
    
    

if  os.path.isfile(kmodel_dir1 + 'modelt1.pt'):
    model1 = torch.load(kmodel_dir1 + 'modelt1.pt')
    
model1.eval()

# Lists to keep track of progress
img_list = []
G_losses = []
G_mse_losses = []
G_fft_mse_losses = []
val_G_losses_mean = []
val_G_mse_losses_mean = []
val_G_fft_losses_mean = []


PSNR = []
SSIM = []
MSE_FFT = []
MSE_IMG = []
NMSE = []
iter_T = []


Pred_slice = 1
Pred_slice_feed = 0
image_fft_r, image_fft_i, image, LayerNum, PosEncoding  = next(iter(dataloader_val))

mask = torch.tensor(mask).to(device)
mask_c = torch.tensor(mask_c).to(device)


T1_pred = infer_using_KspaceNet(image_fft_r,image_fft_i,LayerNum,model=model1,mask=mask,mask_c=mask_c,t_c=4,t_p=1,PosEncoding=PosEncoding.to(device))
T2_pred = infer_using_KspaceNet(image_fft_r,image_fft_i,LayerNum,model=model2,mask=mask,mask_c=mask_c,t_c=1,t_p=4,PosEncoding=PosEncoding.to(device))
imshow(vutils.make_grid((T1_pred).detach().cpu(), padding=2, nrow=5, normalize=False), batch_size, 'T1_pred', cmap = 'gray')
imshow(vutils.make_grid((T2_pred).detach().cpu(), padding=2, nrow=5, normalize=False), batch_size, 'T2_pred', cmap = 'gray')
fix_bad_imgs  = torch.concat((T1_pred,T2_pred),dim=1).cuda()
fix_good_imgs = image[:,Pred_slice,:,:][:,np.newaxis,:,:].cuda()
clip = torch.ones_like(fix_good_imgs).to(device) * 0.1
imshow(vutils.make_grid(torch.minimum(clip,torch.abs(fix_good_imgs-T1_pred)).detach().cpu(), padding=2, nrow=5, normalize=False), batch_size, 'T1_pred_diff', cmap = 'jet')
imshow(vutils.make_grid(fix_good_imgs.detach().cpu(), padding=2, nrow=5, normalize=True), batch_size, 'ground_truth', cmap = 'gray')
import time
intialTime = time.time()

t_iters = 0
epoch = 0 
model1.train(False)
model2.train(False)
netG.train(False)

for i, data in enumerate(dataloader_val, 0):

    image_fft_r, image_fft_i, image, LayerNum, PosEncoding = data
    image = image.to(device)
    image_fft_i = image_fft_i.to(device)
    image_fft_r = image_fft_r.to(device)

    
    T1_pred = infer_using_KspaceNet(image_fft_r,image_fft_i,LayerNum,model=model1,mask=mask,mask_c=mask_c,t_c=4,t_p=1,PosEncoding=PosEncoding.to(device))
    T2_pred = infer_using_KspaceNet(image_fft_r,image_fft_i,LayerNum,model=model2,mask=mask,mask_c=mask_c,t_c=1,t_p=4,PosEncoding=PosEncoding.to(device))

    netG.zero_grad()

    bad_imgs  = torch.concat((T1_pred,T2_pred),dim=1).cuda()
    good_imgs = image[:,Pred_slice,:,:][:,np.newaxis,:,:].cuda()

    fake = netG(bad_imgs.float()).to(device) + bad_imgs[:,Pred_slice_feed,:,:][:,np.newaxis,:,:]
    
    errG_fft, errG_mse, fake= cal_losses(fake,good_imgs,Pred_slice)
    errG =  errG_mse + errG_fft

    for k in range(fake.shape[0]):
        fake = fake.detach()
        good_imgs = good_imgs.detach()
        errG_numerator = torch.sqrt(torch.sum(torch.square(fake[k,:,:,:] - good_imgs[k,:,:,:])))
        errG_denominator = torch.sqrt(torch.sum(torch.square(good_imgs[k,:,:,:])))
        errG_mse = (errG_numerator/errG_denominator)
        MSE_IMG.append(errG_mse.detach().item())
        MSE_FFT.append(errG_fft.detach().item())
        PSNR.append(psnr(fake[k,:,:,:][np.newaxis,:,:,:], good_imgs[k,:,:,:][np.newaxis,:,:,:]).cpu())
        SSIM.append(ssim(fake[k,:,:,:][np.newaxis,:,:,:], good_imgs[k,:,:,:][np.newaxis,:,:,:]).cpu())
        NMSE.append(errG_mse.detach().item())
        iter_T.append(t_iters)
        print('Elapsed time = {}'.format(time.time() - intialTime))
        intialTime = time.time()
        print('[%d/%d][%d/%d]\tLoss_G: %.4f, rrG_mse: %.4f, errG_fft_mse: %.4f, SSIM: %.4f, PSNR: %.4f' % (k, fake.shape[0], i,
                    len(dataloader_val), errG.item(), MSE_IMG[-1], MSE_FFT[-1], SSIM[-1], PSNR[-1]))
        mdic = {"PSNR": PSNR,"SSIM": SSIM,"NMSE": NMSE,"MSE_FFT": MSE_FFT,"MSE_IMG": MSE_IMG,'iter_T':iter_T}

        savemat(save_to_dir + "Training_Losses.mat", mdic)

                # Check how the generator is doing by saving G's output on fixed_noise
        if len(PSNR) > 1:
            if (PSNR[-1] > np.max(PSNR[0:-1])) or (PSNR[-1] < np.min(PSNR[0:-1])):
                imshow(vutils.make_grid((good_imgs[k,:,:,:][np.newaxis,:,:,:]).cpu(), padding=2, nrow=5, normalize=False),
                                        batch_size, '{}_{}_{:.4f}_{:.4f}_GT'.format(k, i, PSNR[-1], SSIM[-1]), cmap = 'gray')
                imshow(vutils.make_grid((fake[k,:,:,:][np.newaxis,:,:,:]).cpu(), padding=2, nrow=5, normalize=False),
                                        batch_size, '{}_{}_{:.4f}_{:.4f}_pred'.format(k, i, PSNR[-1], SSIM[-1]), cmap = 'gray')
                imshow(vutils.make_grid(torch.abs(good_imgs[k,:,:,:][np.newaxis,:,:,:]-fake[k,:,:,:][np.newaxis,:,:,:]).cpu(), padding=2, nrow=5, normalize=False),
                                        batch_size, '{}_{}_{:.4f}_{:.4f}_diff'.format(k, i, PSNR[-1], SSIM[-1]), cmap = 'gray')
                imshow(vutils.make_grid(torch.abs(good_imgs[k,:,:,:][np.newaxis,:,:,:]-fake[k,:,:,:][np.newaxis,:,:,:]).cpu(), padding=2, nrow=5, normalize=False),
                                        batch_size, '{}_{}_{:.4f}_{:.4f}_diff_10'.format(k, i, PSNR[-1], SSIM[-1]), cmap = 'jet')
                imshow(vutils.make_grid(torch.minimum(clip[k,:,:,:][np.newaxis,:,:,:],torch.abs(good_imgs[k,:,:,:][np.newaxis,:,:,:]-fake[k,:,:,:][np.newaxis,:,:,:])).cpu(), padding=2, nrow=5, normalize=False),
                                        batch_size, '{}_{}_{:.4f}_{:.4f}_diff_01'.format(k, i, PSNR[-1], SSIM[-1]), cmap = 'jet')                                     
    t_iters += 1
